In [1]:
import numpy as np

1.1. Реализация функции REF(), приводящей матрицу к ступенчатому виду

In [2]:
def REF(matrix):
    rows, cols = matrix.shape
    lead = 0

    for r in range(rows):
        if lead >= cols:
            break

        i = r
        while matrix[i, lead] == 0:
            i += 1
            if i == rows:
                i = r
                lead += 1
                if cols == lead:
                    return matrix[:r]

        matrix[[i, r]] = matrix[[r, i]]

        for i in range(r + 1, rows):
            if matrix[i, lead] == 1:
                matrix[i] ^= matrix[r]

        lead += 1

    return matrix[:r]

In [3]:
matrix = np.array([[3, 0, 2, 4, 5],
                  [0, 0, 5, 6, 7],
                  [0, 8, 3, -2, 5],
                  [0, 1, 0, 1, 0]])

REF(matrix)

array([[ 3,  0,  2,  4,  5],
       [ 0,  8,  3, -2,  5],
       [ 0,  0,  5,  6,  7]])

1.2. Реализация функции PREF(), которая приводит матрицу к приведённому ступенчатому виду

In [4]:
def RREF(matrix):
    rows, columns = matrix.shape
    pivot_column = 0

    for row_index in range(rows):
        if pivot_column >= columns:
            return matrix

        swap_row = row_index
        while matrix[swap_row, pivot_column] == 0:
            swap_row += 1
            if swap_row == rows:
                swap_row = row_index
                pivot_column += 1
                if pivot_column == columns:
                    return matrix


        matrix[[swap_row, row_index]] = matrix[[row_index, swap_row]]


        if matrix[row_index, pivot_column] != 0:
            matrix[row_index] = matrix[row_index] / matrix[row_index, pivot_column]


        for row in range(rows):
            if row != row_index and matrix[row, pivot_column] == 1:
                matrix[row] = np.logical_xor(matrix[row], matrix[row_index]).astype(int)

        pivot_column += 1

    return matrix


In [5]:
matrix = np.array([[3, 2, 1],
                   [5, 4, 4],
                   [7, 6, 6]])

RREF(matrix)

array([[1, 0, 0],
       [0, 0, 0],
       [1, 1, 1]])

1.3.1. Формирование порождающей матрицы в ступенчатом на основе входной

In [6]:
A = np.array([
    [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]])

A_REF = REF(A)
A_REF

array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

1.3.2 Задать n равное числу столбцов и k равное числу строк полученной матрицы (без учёта полностью нулевых строк).

In [7]:
non_zero_rows = np.any(A_REF , axis=1)
A_REF_new = A_REF [non_zero_rows]
n = A_REF_new.shape[1]
k = A_REF_new.shape[0]
print('n =', n)
print('k =', k)

n = 11
k = 5


1.3.3. Формирование проверочной матрицы на основе порождающей

In [8]:
# Шаг 1. Сформировать матрицу 𝐆∗ в приведённом ступенчатом виде на основе порождающей.
G = np.array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
              [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
              [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])
G_REF = RREF(G)
print('G* =\n', G_REF)

G* =
 [[1 0 1 0 0 1 0 1 0 1 0]
 [0 0 0 1 0 1 0 0 0 1 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]]


In [9]:
# Шаг 2. Зафиксировать ведущие столбцы lead матрицы 𝐆∗.
def find_leading_columns(matrix):
    rows, cols = matrix.shape
    leading_cols = []
    current_row = 0
    current_col = 0

    while current_row < rows and current_col < cols:

        while current_col < cols and matrix[current_row, current_col] == 0:
            current_col += 1

        if current_col < cols:
            leading_cols.append(current_col)

        current_row += 1
        current_col += 1
    return leading_cols

leading_cols = find_leading_columns(G_REF)
print(f"Номера ведущих столбцов: {leading_cols}")

Номера ведущих столбцов: [0, 3, 4, 6, 8]


In [14]:
# Шаг 3. Сформировать сокращённую матрицу 𝐗, удалив ведущие столбцы матрицы 𝐆∗.
# Удаляем ведущие столбцы, оставляя только остальные
X = np.empty(
    [G_REF.shape[0], G_REF.shape[1]-len(leading_cols)],
    dtype=int
)

pos = 0
for i in range(G_REF.shape[1]):
    if i not in leading_cols:
        X[:, pos] = G_REF[:, i]
        pos += 1

print('X =\n', X)

X =
 [[0 1 1 1 1 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 1 1]]


In [15]:
#Шаг 4. Сформировать матрицу 𝐇, поместив в строки, соответствующие позициям ведущих столбцов строки из 𝐗, а в остальные – строки единичной матрицы.
I = np.eye(max(X.shape))

H = np.empty(
    [X.shape[0]+I.shape[0], X.shape[1]],
    dtype=int
)

posX, posI = 0, 0
for i in range(H.shape[0]):
    if i in leading_cols:
        H[i, :] = X[posX, :]
        posX += 1
    else:
        H[i, :] = I[posI, :]
        posI += 1

print('H =\n', H)

H =
 [[0 1 1 1 1 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


1.4.1. Сложение слов из пораждающего множества с сохранением неповторяющихся

In [16]:
def generate_codewords_by_sum(G):
    from itertools import combinations

    codewords = set()
    for r in range(1, G.shape[0] + 1):
        for subset in combinations(G, r):
            codeword = np.bitwise_xor.reduce(subset) % 2
            codewords.add(tuple(codeword))
    return np.array(list(codewords))

In [17]:
print("Кодовые слова (способ 1):")
print(generate_codewords_by_sum(G))

Кодовые слова (способ 1):
[[0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 1 1 1 1 1 1 1 1]
 [1 0 1 0 1 1 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 1 0 1]
 [1 0 1 1 1 0 1 0 0 1 0]
 [1 0 1 1 0 0 1 1 1 0 0]
 [0 0 0 0 1 0 0 1 1 1 0]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 1 1 1 1 1 0 0 0]
 [0 0 0 0 1 0 1 1 1 0 0]
 [1 0 1 1 0 0 1 1 0 1 1]
 [0 0 0 1 1 1 0 1 1 0 1]
 [1 0 1 0 0 1 1 1 0 0 0]
 [1 0 1 0 0 1 1 1 1 1 1]
 [0 0 0 1 0 1 0 0 1 0 0]
 [0 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [1 0 1 0 1 1 0 0 0 1 1]
 [1 0 1 0 0 1 0 1 0 1 0]
 [1 0 1 1 1 0 0 0 1 1 1]
 [1 0 1 1 1 0 0 0 0 0 0]
 [1 0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 1 0 0 0 1]
 [1 0 1 0 1 1 1 0 1 1 0]
 [1 0 1 1 0 0 0 1 1 1 0]
 [1 0 1 0 1 1 1 0 0 0 1]
 [0 0 0 1 0 1 1 0 1 1 0]
 [1 0 1 0 0 1 0 1 1 0 1]
 [0 0 0 0 0 0 0 0 1 1 1]
 [1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 0 1 0 0 0 1 1]]


In [18]:
def generate_codewords_by_multiplication(G):
    k = G.shape[0]
    from itertools import product

    codewords = set()
    for binary_word in product([0, 1], repeat=k):
        codeword = np.dot(binary_word, G) % 2
        codewords.add(tuple(codeword))
    return np.array(list(codewords))

In [19]:
print("Кодовые слова (способ 2):")
print(generate_codewords_by_multiplication(G))

Кодовые слова (способ 2):
[[0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 1 1 1 1 1 1 1 1]
 [1 0 1 0 1 1 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 1 0 1]
 [1 0 1 1 1 0 1 0 0 1 0]
 [1 0 1 1 0 0 1 1 1 0 0]
 [0 0 0 0 1 0 0 1 1 1 0]
 [0 0 0 1 1 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 0 1 0 1 1 1 0 0]
 [1 0 1 1 0 0 1 1 0 1 1]
 [0 0 0 1 1 1 0 1 1 0 1]
 [1 0 1 0 0 1 1 1 0 0 0]
 [1 0 1 0 0 1 1 1 1 1 1]
 [0 0 0 1 0 1 0 0 1 0 0]
 [0 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [1 0 1 0 1 1 0 0 0 1 1]
 [1 0 1 0 0 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 1 1 1]
 [1 0 1 1 1 0 0 0 0 0 0]
 [1 0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 1 0 0 0 1]
 [1 0 1 0 1 1 1 0 1 1 0]
 [1 0 1 1 0 0 0 1 1 1 0]
 [0 0 0 1 0 1 1 0 1 1 0]
 [1 0 1 0 1 1 1 0 0 0 1]
 [1 0 1 0 0 1 0 1 1 0 1]
 [0 0 0 0 0 0 0 0 1 1 1]
 [1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 0 1 0 0 0 1 1]]


In [20]:
def check_codewords_with_H(codewords, H):
    for word in codewords:
        syndrome = np.dot(word, H) % 2
        if np.any(syndrome != 0):
            return False
    return True

In [21]:
codewords = generate_codewords_by_multiplication(G)

is_valid = check_codewords_with_H(codewords, H)

print("Кодовые слова корректны:", is_valid)

Кодовые слова корректны: True


1.5. Вычисление кодового расстояния получившегося кода

In [22]:
def hamming_distance(word1, word2):
    return np.sum(np.array(word1) != np.array(word2))

def find_code_distance(codewords):
    min_distance = np.inf
    for i in range(len(codewords)):
        for j in range(i + 1, len(codewords)):
            dist = hamming_distance(codewords[i], codewords[j])
            if 0 < dist < min_distance:
                min_distance = dist
    return min_distance

In [23]:
d = find_code_distance(codewords)
t = d - 1
print("Кодовое расстояние d:", d)
print("Кратность обнаруживаемой ошибки t:", t)

Кодовое расстояние d: 2
Кратность обнаруживаемой ошибки t: 1


In [24]:
def introduce_error(codeword, error_positions):
    erroneous_word = np.array(codeword, dtype=int)
    for pos in error_positions:
        erroneous_word[pos] = (erroneous_word[pos] + 1) % 2  # Внесение ошибки (переворот бита)
    return erroneous_word

def detect_error_with_H(codeword, H):
    syndrome = np.dot(codeword, H) % 2
    return syndrome

1.5.1. Внесение в кодовое слово ошибки кратности не более t, умножение полученное слово на H, проверка обнаружения ошибки.

In [25]:
v = codewords[4]  # Берем кодовое слово
t = 1  # Кратность ошибки

error_positions_t = [2]
v_with_error_t = introduce_error(v, error_positions_t)

# Проверяем обнаружение ошибки
syndrome_t = detect_error_with_H(v_with_error_t, H)
print("Кодовое слово с ошибкой (кратности t):", v_with_error_t)
print("Синдром ошибки (кратность t):", syndrome_t)
print('\n-> '+("Ошибка обнаружена" if np.any(syndrome_t) else "Ошибка не обнаружена"))

Кодовое слово с ошибкой (кратности t): [1 0 0 1 1 0 1 0 0 1 0]
Синдром ошибки (кратность t): [0 1 0 0 0 0]

-> Ошибка обнаружена


1.5.2. Поиск для некоторого кодового слова ошибки кратности t+1 такую, что при умножении на H ошибка не будет обнаружена

In [26]:
t_plus_1 = t + 1
error_positions_t_plus_1 = [0, 2, 3, 4]

v_with_error_t_plus_1 = introduce_error(v, error_positions_t_plus_1)

# Проверяем обнаружение ошибки
syndrome_t_plus_1 = detect_error_with_H(v_with_error_t_plus_1, H)
print("Кодовое слово с ошибкой (кратности t+1):", v_with_error_t_plus_1)
print("Синдром ошибки (кратность t+1):", syndrome_t_plus_1)
print('\n-> '+("Ошибка обнаружена" if np.any(syndrome_t_plus_1) else "Ошибка не обнаружена"))

Кодовое слово с ошибкой (кратности t+1): [0 0 0 0 0 0 1 0 0 1 0]
Синдром ошибки (кратность t+1): [0 0 0 0 0 0]

-> Ошибка не обнаружена
